In [2]:
import pandas as pd
import numpy as np
import datetime
import requests
import warnings

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.dates as mdates
import seaborn as sns
# import squarify
import plotly.offline as py
# import plotly_express as px

# from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
# from fbprophet import Prophet
# from fbprophet.plot import plot_plotly, add_changepoints_to_plot

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [4]:

from google.colab import files 
  
  
uploaded = files.upload()


Saving finalDataset.csv to finalDataset.csv


In [5]:
cdata= pd.read_csv('finalDataset.csv', index_col=0)
cdata.head()

,country,sub-region,date,driving,transit,walking,Confirmed,Deaths,Recovered
0,Albania,Total,2020-01-22,-6.49,NaN,-4.61,0,0,0
1,Albania,Total,2020-01-23,-7.06,NaN,-5.76,0,0,0
2,Albania,Total,2020-01-24,2.13,NaN,-6.27,0,0,0
3,Albania,Total,2020-01-25,2.38,NaN,-2.94,0,0,0
4,Albania,Total,2020-01-26,1.41,NaN,-22.73,0,0,0


In [6]:
cdata['date'] =  pd.to_datetime(cdata['date'], infer_datetime_format=True)
start_date = cdata['date'].iloc[-1]
start_date = start_date + pd.DateOffset(1)

In [7]:
cdata['date'] =  pd.to_datetime(cdata['date'], infer_datetime_format=True)
cdata = cdata.set_index('date')
cdata.index

DatetimeIndex(['2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25',
               '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29',
               '2020-01-30', '2020-01-31',
               ...
               '2020-09-14', '2020-09-15', '2020-09-15', '2020-09-15',
               '2020-09-16', '2020-09-16', '2020-09-16', '2020-09-17',
               '2020-09-17', '2020-09-17'],
              dtype='datetime64[ns]', name='date', length=252663, freq=None)

In [8]:
df = cdata.head(10)

In [9]:
confirmed_d = cdata['Confirmed'].resample('MS').mean()
deaths_d = cdata['Deaths'].resample('MS').mean()
recovered_d= cdata['Recovered'].resample('MS').mean()
driving_d= cdata['driving'].resample('MS').mean()
walking_d= cdata['transit'].resample('MS').mean()
transit_d= cdata['walking'].resample('MS').mean()

In [10]:
def scoring_rmse(y_pred, y_test):
    print(y_pred)
    print(y_test)
    rmse = np.sqrt(np.mean(np.square(y_test.values - y_pred.values)))
    return rmse

def scoring_mape(pred,test):
    try:
        mape = np.mean(np.abs((test - pred) / test))
    except:
        mape = -1
    return mape

In [11]:
def data_creation_sarima(data,division):
    """
    return
    -----------------
    split[0]: training set
    split[1]: test set
    """
    split = []
    div = division
    data.fillna(method='ffill',inplace=True)
    train_size = int(len(data) * div)
    train, test = data[0:train_size], data[train_size:len(data)]
    split.append(train)
    split.append(test)
    return(split)

In [12]:
def sarimax_modelling2(data,exog,div=0.9,max_range=1):
    seas_values = [12]
    result_list = []
    train_size = int(len(data) * div)
    min_rmse = 999999
    mape = 99999
    param = None
    seas_param = None
    train_exog,test_exog = exog[0:train_size], exog[train_size:len(data)]
    for i in seas_values:
#         try:
#             result_list.append(get_sarimax(data,train_exog,test_exog,max_range,seasonal_factor=i))
        temp_res = get_sarimax(data,train_exog,test_exog,max_range,seasonal_factor=i)
        temp_rmse = temp_res['rmse']
        if temp_rmse<min_rmse:
            min_rmse = temp_rmse
            param = temp_res['parameter']
            seas_param = temp_res['seasonal paramater']
            mape = temp_res['mape']
#         except:
#             pass
    return {'param':param,'seas_param':seas_param,'rmse':min_rmse,'mape':mape}

In [13]:
def get_sarimax(trend,exog_train,exog_test,max_range=3,seasonal_factor=12):
    #sarima modelling on trend with period = 12
    train,test = data_creation_sarima(trend,0.9)
    param_order = []
    param_seas = []
    p = d = q = range(0, max_range)
    pdq = list(itertools.product(p, d, q))

    #seasonal parameter = seasonal_factor
    seasonal_pdq = [(x[0], x[1], x[2], seasonal_factor) for x in list(itertools.product(p, d, q))]
#     print(pdq,seasonal_pdq)
    
    min_rmse=9999999
    min_model_mape = 9999999
    min_model_aic = 9999999
    final_predictions=-1 
    result_param=-1 
    result_param_seasonal=-1
    for i in [(1,1,1)]:#range(0,len(pdq)):
        for j in [(1,0,0,12)]:#range(0,len(seasonal_pdq)):
#             try:

            param = pdq[i]
            param_seasonal = seasonal_pdq[j]
            mod = sm.tsa.statespace.SARIMAX(train,exog=exog_train,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False)
            results = mod.fit()
            aic_value = results.aic
#                 if math.isnan(temp) or math.isinf(temp):
#                     temp = 999.99
            pred = results.predict(len(train),len(trend)-1,exog = exog_test)
            rmse = scoring_rmse(pred,test)
            mape = scoring_mape(pred,test)
#                 print(pred,rmse,mape)
            if rmse < min_rmse:
                min_rmse = rmse
                min_model_aic = aic_value
                min_model_mape = mape
                result_param = param
                result_param_seasonal = param_seasonal
                final_predictions = pred

#             except:
#                 continue
                    
    return {'rmse':min_rmse, 'mape':min_model_mape, 'aic':min_model_aic, 'prediction':final_predictions, 'parameter':result_param, 'seasonal paramater':result_param_seasonal}


In [14]:
def fcast_sarimax(arima,exog_forecast,n):
    #forecasts
    pred_uc = arima.get_forecast(steps=n,exog=exog_forecast)
    forecast = pred_uc.predicted_mean
    print(forecast.tail(12))

    #plotting
#     pred_ci = pred_uc.conf_int()
#     ax = df.plot(label='observed', figsize=(14, 4))
#     pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
#     ax.fill_between(pred_ci.index,
#                     pred_ci.iloc[:, 0],
#                     pred_ci.iloc[:, 1], color='k', alpha=.25)
#     ax.set_xlabel('Month')
#     ax.set_ylabel('Revenue')
#     plt.legend()
#     plt.show()
    return(forecast.tail(n))

In [15]:
def sarimax(train,exog_train,p,d,q,P,D,Q,M,summary=True):
    arima = sm.tsa.statespace.SARIMAX(train,exog=exog_train,order=(p,d,q),seasonal_order=(P,D,Q,M),enforce_stationarity=False, enforce_invertibility=False).fit()
    if summary==True:
        arima.summary()
    return(arima)

In [21]:
data = cdata['Confirmed'].astype(float)
# exog = cdata[['driving','transit','walking'],'date']
exog = cdata[['driving','transit','walking']].fillna(method='ffill').fillna(0).astype(float)

p,d,q,P,D,Q,M = 1,1,1,1,0,0,12
model = sarimax(data,None,p,d,q,P,D,Q,M,summary=False)

#need to create the exog_forecast dataframe which contains exogenous variables for future time
# forecast = fcast_sarimax(model,None,10)
exog_list=['driving','walking','transit']
exog_df = pd.DataFrame()
for var in exog_list:
    data_ = cdata[var]
    model_ = sarimax(data_,None,p,d,q,P,D,Q,M,summary=False)
    forecast_ = fcast_sarimax(model_,None,30)
    exog_df[var] = forecast_

# loop ended here
#replace exog in last line with exog_df and 10 by 30

forecast = fcast_sarimax(model,exog_df,30)




/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:576: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.



252681   -4.468744
252682   -4.468196
252683   -4.467089
252684   -4.467676
252685   -4.468886
252686   -4.467015
252687   -4.467554
252688   -4.467656
252689   -4.467649
252690   -4.467659
252691   -4.467658
252692   -4.467641
dtype: float64


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:576: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.



252681    12.010090
252682    12.010503
252683    12.010635
252684    12.010673
252685    12.010682
252686    12.010995
252687    12.010900
252688    12.010849
252689    12.010814
252690    12.010798
252691    12.010787
252692    12.010780
dtype: float64


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:576: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.



252681    21.645465
252682    21.645465
252683    21.645465
252684    21.645465
252685    21.645465
252686    21.645465
252687    21.645465
252688    21.645465
252689    21.645465
252690    21.645465
252691    21.645465
252692    21.645465
dtype: float64


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:1935: ValueWarning:

Exogenous array provided to predict, but additional data not required. `exog` argument ignored.

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:576: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.



252681    1064.041985
252682    1064.042026
252683    1064.042038
252684    1064.250328
252685    1064.250329
252686    1064.250329
252687    1064.171125
252688    1064.148922
252689    1064.142698
252690    1064.140953
252691    1064.140464
252692    1064.140327
dtype: float64


In [28]:

end_date = start_date + pd.DateOffset(30)
datelist = pd.date_range(start_date, periods=30).tolist()
final_df = pd.DataFrame(list(zip(forecast, datelist)), 
               columns =['forecast', 'date']) 
# final_df['date'] = pd.date_range(start_date, periods = 30, freq ='H')
# final_df.to_json()
# print(forecast)
# end_date
# datelist 
final_df.head()

,forecast,date
0,1064.859206,2020-09-18
1,1064.539412,2020-09-19
2,1064.449764,2020-09-20
3,1064.424634,2020-09-21
4,1064.417589,2020-09-22


In [29]:
final_df.tail()

,forecast,date
25,1064.148922,2020-10-13
26,1064.142698,2020-10-14
27,1064.140953,2020-10-15
28,1064.140464,2020-10-16
29,1064.140327,2020-10-17


## Country specific code 

**ALBANIA**

In [22]:
data_req = cdata.loc[cdata['country']=='Albania']
data = data_req['Confirmed'].astype(float)
# exog = cdata[['driving','transit','walking'],'date']
exog = data_req[['Confirmed','transit','walking']].fillna(method='ffill').fillna(0).astype(float)

p,d,q,P,D,Q,M = 1,1,1,1,0,0,12
model = sarimax(data,None,p,d,q,P,D,Q,M,summary=False)

#need to create the exog_forecast dataframe which contains exogenous variables for future time
# forecast = fcast_sarimax(model,None,10)
exog_list=['driving','walking','transit']
exog_df = pd.DataFrame()
for var in exog_list:
    data_ = cdata[var]
    model_ = sarimax(data_,None,p,d,q,P,D,Q,M,summary=False)
    forecast_ = fcast_sarimax(model_,None,30)
    exog_df[var] = forecast_

# loop ended here
#replace exog in last line with exog_df and 10 by 30

forecast_drive = fcast_sarimax(model,exog_df,30)
# forecast_drive['date'] = pd.date_range(start_date, periods = 30, freq ='H')
# forecast_drive.to_json()
# forecast_drive

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:576: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.



252681   -4.468744
252682   -4.468196
252683   -4.467089
252684   -4.467676
252685   -4.468886
252686   -4.467015
252687   -4.467554
252688   -4.467656
252689   -4.467649
252690   -4.467659
252691   -4.467658
252692   -4.467641
dtype: float64


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:576: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.



252681    12.010090
252682    12.010503
252683    12.010635
252684    12.010673
252685    12.010682
252686    12.010995
252687    12.010900
252688    12.010849
252689    12.010814
252690    12.010798
252691    12.010787
252692    12.010780
dtype: float64


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:576: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.



252681    21.645465
252682    21.645465
252683    21.645465
252684    21.645465
252685    21.645465
252686    21.645465
252687    21.645465
252688    21.645465
252689    21.645465
252690    21.645465
252691    21.645465
252692    21.645465
dtype: float64
256    14827.707622
257    14987.073603
258    15147.255302
259    15308.255925
260    15470.080154
261    15632.731932
262    15796.217214
263    15960.539430
264    16125.702861
265    16291.711819
266    16458.570633
267    16626.283654
dtype: float64


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:1935: ValueWarning:

Exogenous array provided to predict, but additional data not required. `exog` argument ignored.

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:576: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.



238                                               12092.8
239                                               12238.3
240                                               12384.5
241                                               12531.4
242                                               12679.2
243                                               12827.7
244                                               12977.1
245                                               13127.2
246                                                 13278
247                                               13429.5
248                                               13581.8
249                                               13734.6
250                                               13888.4
251                                                 14043
252                                               14198.3
253                                               14354.5
254                                               14511.4
255           

In [31]:
# forecast_drive
datelist_country = pd.date_range(start_date, periods=30).tolist()
final_df_country = pd.DataFrame(list(zip(forecast_drive, datelist)), 
               columns =['forecast', 'date']) 
final_df_country.head()

,forecast,date
0,12092.770294,2020-09-18
1,12238.258554,2020-09-19
2,12384.451896,2020-09-20
3,12531.420673,2020-09-21
4,12679.177006,2020-09-22


In [32]:
final_df_country.to_json()

'{"forecast":{"0":12092.7702944016,"1":12238.2585538878,"2":12384.4518960767,"3":12531.4206732342,"4":12679.1770062001,"5":12827.7247137247,"6":12977.0509910283,"7":13127.1680175998,"8":13278.0380634405,"9":13429.5485245474,"10":13581.7615692006,"11":13734.6478487999,"12":13888.4174480454,"13":14042.9740509788,"14":14198.32154874,"15":14354.4645455783,"16":14511.4071817413,"17":14669.1535490693,"18":14827.7076218542,"19":14987.0736031865,"20":15147.2553018312,"21":15308.2559245917,"22":15470.0801538599,"23":15632.7319317883,"24":15796.2172144147,"25":15960.5394295927,"26":16125.7028607048,"27":16291.7118188297,"28":16458.5706330589,"29":16626.2836540664},"date":{"0":1600387200000,"1":1600473600000,"2":1600560000000,"3":1600646400000,"4":1600732800000,"5":1600819200000,"6":1600905600000,"7":1600992000000,"8":1601078400000,"9":1601164800000,"10":1601251200000,"11":1601337600000,"12":1601424000000,"13":1601510400000,"14":1601596800000,"15":1601683200000,"16":1601769600000,"17":16018560000

In [33]:
data_req = cdata.loc[cdata['country']=='Albania']
data_req

,country,sub-region,driving,transit,walking,Confirmed,Deaths,Recovered
date,,,,,,,,
2020-01-22,Albania,Total,-6.49,NaN,-4.61,0,0,0
2020-01-23,Albania,Total,-7.06,NaN,-5.76,0,0,0
2020-01-24,Albania,Total,2.13,NaN,-6.27,0,0,0
2020-01-25,Albania,Total,2.38,NaN,-2.94,0,0,0
2020-01-26,Albania,Total,1.41,NaN,-22.73,0,0,0
...,...,...,...,...,...,...,...,...
2020-09-13,Albania,Total,73.76,NaN,54.67,11353,334,6569
2020-09-14,Albania,Total,42.75,NaN,59.40,11520,338,6615
2020-09-15,Albania,Total,44.04,NaN,57.19,11672,340,6668


# Warning Cell: Don't run after this cell

### The below one is useless, just the above one will do

In [ ]:
# pred = results.get_prediction(start=pd.to_datetime('2020-09-17'), dynamic=False)
# pred_ci = pred.conf_int()

# ax = confirmed_d['2020':].plot(label='observed')
# pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(14, 7))ax.fill_between(pred_ci.index,
#                 pred_ci.iloc[:, 0],
#                 pred_ci.iloc[:, 1], color='k', alpha=.2)
# ax.set_xlabel('Date')
# ax.set_ylabel('Confirmed Cases')
# plt.legend()
# plt.show()

In [ ]:
# confirmed_forecasted = pred.predicted_mean
# confirmed_truth = confirmed_d['2020-09-17':]mse = ((confirmed_forecasted - confirmed_truth) ** 2).mean()
# print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))

In [ ]:
# pred_uc = results.get_forecast(steps=100)
# pred_ci = pred_uc.conf_int()ax = confirmed_d.plot(label='observed', figsize=(14, 7))
# pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
# ax.fill_between(pred_ci.index,
#                 pred_ci.iloc[:, 0],
#                 pred_ci.iloc[:, 1], color='k', alpha=.25)
# ax.set_xlabel('Date')
# ax.set_ylabel('Confirmed Cases')
# plt.legend()
# plt.show()